Version Date: May 12 2023, Henrik Loecke

In [1]:
#PERMANENT CELL 1

import os
import mikeio
import mikeio1d
from mikeio1d.res1d import Res1D
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ctypes
import traceback
MessageBox = ctypes.windll.user32.MessageBoxA
from Result_Lookup_Variables import *

In [ ]:
colors = ['orchid','lightsalmon','slategrey','purple','green','orange','grey','brown','crimson','tan','firebrick']
node_colors = ['blue','olive','red']
flood_types = ['WaterFlowRateAboveGround','WaterSpillDischarge']

for m in master_list:
    
    model_area = m[0]
    model = m[1]
    result_folder = m[2]
    output_folder = m[3]
    result_list = m[4]
    groupby_acronym_owner = m[5]
    nodes_df = pd.read_csv(result_folder + '\\Connection_Stats_Nodes.csv',dtype={'MUID': str, 'Asset Name': str})
    nodes_df['Asset Name'].fillna('',inplace=True)
    lines_df =  pd.read_csv(result_folder + '\\Connection_Stats_Lines.csv',dtype={'MUID': str, 'Acronym': str, 'Owner': str, 'From Node': str, 'To Node': str})     
    lines_df.Acronym.fillna('',inplace=True)
    
    #Ensure that result_list has design storms next to each other.
    plot_start_indexes = []
    plot_end_indexes = []
    
    plot_headers = []
    r_temp = []
    for r in result_list:
        if not r[2] in plot_headers: #Would normally use set but order does matter
            plot_headers.append(r[2])
    for plot_header in plot_headers:
        for r in result_list:
            if r[2] == plot_header:
                r_temp.append(r)
    result_list = r_temp
    
    #Get start and end indexes for the plots
    previous_plot_header = 'X'
    for i, r in enumerate(result_list):
        if i == 0 or r[2] != previous_plot_header:
            plot_start_indexes.append(i) 
            if i > 0:
                plot_end_indexes.append(i-1)
        if i == len(result_list)-1:
            plot_end_indexes.append(i)
        previous_plot_header = r[2]

    for r in result_list:
        header = r[0]
        file = r[1]
        res1d = Res1D(result_folder + '\\' + file)
        r.append(res1d)
#         try:
#             res1d = Res1D(result_folder + '\\' + file[:-6] + '.ADDOUT.res1d')
#             r.append(res1d)
#         except:
#             print('WARNING! No .ADDOUT file was found for ' + file)
                        
        if os.path.exists(result_folder + '\\' + file[:-6] + '.ADDOUT.res1d'):
#             print ("Importing spilling " + file[:-6] + ".ADDOUT.res1d at " + str(dt.datetime.now()))
            res1d = Res1D(result_folder + '\\' + file[:-6] + '.ADDOUT.res1d')
            r.append(res1d)
        elif flood_types[0] in res1d.quantities or flood_types[1] in res1d.quantities:
            pass #Maintain same res1d file which has flooding (MIKE+)    
            flooding_result_available = True
            r.append(res1d) #Same result file as for the WL and Q
        else:
            print('WARNING! ' + file[:-6] + '.ADDOUT.res1d does not exist and flooding not available in ' + file + '.')

    
    result_folder = m[2]
    coordinate_df = pd.read_csv(result_folder + '\\Google_Coordinates.csv',dtype={'MUID': str}).set_index('MUID')
    
    class Element_Types:
        def __init__(self,el_type,df,quantity,axis_title,decimals):
            self.el_type = el_type
            self.df = df
            self.quantity = quantity
            self.axis_title = axis_title
            self.decimals = decimals

    Node = Element_Types('Node',nodes_df,'Water Level','Water Level (m)',2)
    Line = Element_Types('Link',lines_df,'Discharge','Discharge (L/s)',1)

    ETs = [Node,Line]
    
    for ET in ETs:
        
        
        for index, row in ET.df.iterrows():
            
            plots = []
            muid = row['MUID'].strip()
            
            stats = []
            flood_stats = []

            if ET.el_type == 'Node':
                upstream_df = lines_df[lines_df.ToNode==muid]
                upstream_lines = list(upstream_df.MUID)
                upstream_layers = list(upstream_df.Layer)
                upstreams = list(zip(upstream_lines,upstream_layers))

                downstream_df = lines_df[lines_df.FromNode==muid]
                downstream_lines = list(downstream_df.MUID)
                downstream_layers = list(downstream_df.Layer)
                downstreams = list(zip(downstream_lines,downstream_layers))
                map_node = muid
                element_type = 'Node'
                asset_name = row['Asset Name'].strip()
                cover_type = row['Cover Type']
                if row['Network Type'] == 3:
                    cover_type = 'Outfall '
                elif row['Cover Type'] == 2:
                    cover_type = 'Sealed '
                else:
                    cover_type = 'Unsealed '

            else:
                from_node = row['FromNode']
                to_node = row['ToNode']
                layer = row['Layer']
                fig = go.Figure()
                if layer == 'Link':
                    muid_adj = muid
                else:
                    muid_adj = layer + ':' + muid 
                map_node = from_node
                element_type = layer
                asset_name = row['Acronym'].strip()
                cover_type = ''
                folder = row['Folder']


            for i, r in enumerate(result_list):


                if i in plot_start_indexes:
                    if ET.el_type == 'Node':
                        fig = make_subplots(specs=[[{"secondary_y": True}]])
                    else:
                        fig = go.Figure()

                    color_index = 0
                    if ET.el_type == 'Node':
                        time_bookends = [r[3].time_index[0],r[3].time_index[-1]]
                        for j, column in enumerate(nodes_df.columns[4:]):
                            if not np.isnan(row[column]):
                                fig.add_trace(go.Scatter(x=time_bookends, 
                                                 y = [row[column],row[column]], 
                                                 mode='lines',line_color=node_colors[j],name=column))   
                                
                previous_plot_header = r[2]
                flood_values = None

                if ET.el_type == 'Node':                    
                    values = r[3].query.GetNodeValues(muid, "WaterLevel")
                    if values != None:
                        values = list(values)
                        try:
                            if cover_type == 'Sealed ' or cover_type == 'Outfall ':
                                flood_stats.append([0,0])
                            else:
                                flood_values_found = False
                                for j, flood_type in enumerate(flood_types):
                                    flood_values = r[4].query.GetNodeValues(muid,flood_type)
                                    if flood_values != None:
                                        flood_values_found = True
                                        flood_values = list(flood_values)
                                        flood_values = [k * 1000 for k in flood_values]
                                        flood_stats.append([round(max(flood_values),1),round(sum(flood_values)*300/1000,0)])
                                if flood_values_found == False:
                                    flood_stats.append([0,0])
                        except:
                            pass
                else:
                    try:
                        values = r[3].query.GetReachEndValues(muid_adj, "Discharge")
                    except:
                        values = None
                    if values != None:
                        values = list(values)
                        values = [k * 1000 for k in values]
                if values != None:
                    stats.append([r[0],round(max(values),ET.decimals)])
                    fig.add_trace(go.Scatter(x=r[3].time_index, 
                                                 y = values, 
                                                 mode='lines',line_color=colors[color_index],name=r[0]))
                    if flood_values != None:
                        fig.add_trace(go.Scatter(x=r[3].time_index, 
                                                 y = flood_values, 
                                                 mode='lines',line_color=colors[color_index],
                                                 line_dash='dash',name=r[0] + ' Spilling'),secondary_y=True)

                color_index += 1
                                
                if i in plot_end_indexes:
                    fig.update_layout(
                        autosize=False,
                        width = 1362,
                        height=500,
                        margin=dict(
                            l=50,
                            r=50,
                            b=25,
                            t=35,
                            pad=4
                            ),
                        yaxis_title=ET.axis_title
                        )
                    if ET.el_type == 'Node':                            
                        fig['layout']['yaxis2']['title']='Manhole Spilling (L/s)'
                        fig.update_layout(yaxis2_rangemode="tozero")

                    if r[2] == 0:
                        plot_header = "<h2>" + r[0] + "</h2>\n"
                    else:
                        plot_header = "<h2>" + r[2] + "</h2>\n"
                    plots.append([plot_header,fig])

            output_subfolders = [output_folder + "\\All_Elements"]
            if groupby_acronym_owner == True:
                if ET.el_type == 'Node':
                    node_line_df = lines_df[['MUID','Folder','FromNode','ToNode']].query("FromNode == '" + muid + "' | ToNode == '" + muid + "'")
                    node_line_df.set_index('MUID',inplace=True)
                    for folder in node_line_df.Folder.unique():
                        output_subfolder = output_folder + "\\" + folder
                        output_subfolders.append(output_subfolder)                       
                else: 
                    output_subfolder = output_folder + "\\" + folder
                    output_subfolders.append(output_subfolder)

            for i, output_subfolder in enumerate(output_subfolders):
                
                relative_output_subfolder = '..\\' + os.path.basename(output_subfolder)

                if not os.path.isdir(output_subfolder): os.makedirs(output_subfolder) 

                with open(output_subfolder + "\\" + model_area + "_" + ET.quantity.replace(' ','') + "_" + element_type + "_" + muid + ".html", 'w') as f:
                    f.write('<link rel="stylesheet" href="..\Maps_And_CSS\style.css">\n')
                    
                    f.write('<!DOCTYPE html>')
                    f.write('<html>')
                    f.write('<head>')
                    f.write('<meta charset="utf-8">')
                    f.write('</head>')
                    f.write('<body>')
                    
                    f.write('<div class="sidenav">\n')

                    f.write("<h2>Links to Connected Elements</h2>\n")

                    if ET.el_type == 'Node':
                        for upstream in upstreams:
                            if i == 0:
                                link_sub_folder = relative_output_subfolder
                            else:
                                link_sub_folder = relative_output_subfolder + "\\" + node_line_df.loc[upstream[0],'Folder']
                            path = link_sub_folder + "\\" + model_area + "_Discharge_" + upstream[1] + "_" + upstream[0] + ".html"
                            f.write('<a href="' + path + '">Upstream ' + upstream[1] + ' ' + upstream[0] + '</a>\n')
                            f.write("<br>")
                        for downstream in downstreams:
                            if i == 0:
                                link_sub_folder = relative_output_subfolder
                            else:
                                link_sub_folder = relative_output_subfolder + "\\" + node_line_df.loc[downstream[0],'Folder']
                            path = link_sub_folder + "\\" + model_area + "_Discharge_" + downstream[1] + "_" + downstream[0] + ".html"
                            f.write('<a href="' + path + '">Downstream ' + downstream[1] + ' ' + downstream[0] + '</a>\n')
                            f.write("<br>")
                    else:  
                        path = relative_output_subfolder + "\\" + model_area + "_WaterLevel_Node_" + from_node + ".html"
                        f.write('<a href="' + path + '">Upstream Node ' + from_node + '</a>\n')
                        f.write("<br>")

                        try:
                            path = relative_output_subfolder + "\\" + model_area + "_WaterLevel_Node_" + to_node + ".html"
                            f.write('<a href="' + path + '">Downstream Node ' + to_node + '</a>\n')
                        except:
                            f.write('<p>There is no downstream node.</p>\n')
                        f.write("<br>")
                        
                    f.write("<h2>Map</h2>\n")                   
                    map_string = '<img src="..\\Maps_And_CSS\\' + map_node + '.jpg" alt="' + map_node + '">\n'
                    f.write(map_string + "\n")
                    f.write("<p></p>\n")
                    google_map_string = '<a href="https://maps.google.com/?q='
                    google_map_string += str(coordinate_df.loc[map_node,'Y']) + ', '
                    google_map_string += str(coordinate_df.loc[map_node,'X']) + '" '
                    google_map_string += 'target="_blank">View in Google Maps (Pipes not shown)</a>'
                    f.write(google_map_string + "\n")

                    #Statistics
                    f.write("<h2>Statistics</h2>\n")                               
                    f.write("<table>\n")
                    f.write("  <tr>\n")
                    f.write("    <th>Description</th>\n")
                    if ET.el_type == 'Node':
                        f.write("    <th>HGL (m)    </th>\n")
                        f.write("    <th>Spill (L/s)</th>\n")
                        f.write("    <th>Spill (m<sup>3</sup>)</th>\n")
                    else:
                        f.write("    <th>Maximum " + ET.axis_title + "</th>\n")
                    f.write("  </tr>\n")
                    if ET.el_type == 'Node':
                        f.write("  <tr>\n")
                        f.write("    <td>Safe Operating Head</td>\n")
                        if np.isnan(row['Safe Operating Head']):
                            f.write("    <td>None</td>\n")
                        else:
                            f.write("    <td>" + str(round(row['Safe Operating Head'],2)) + "</td>\n")
                        f.write("    <td></td>\n")
                        f.write("    <td></td>\n")
                        f.write("  </tr>\n")                    
                    for j, stat in enumerate(stats):
                        f.write("  <tr>\n")
                        f.write("    <td>" + stat[0] + "</td>\n")  
                        f.write("    <td>" + str(stat[1]) + "</td>\n")
                        if ET.el_type == 'Node':
                            f.write("    <td>" + str(flood_stats[j][0]) + "</td>\n")
                            f.write("    <td>" + str(flood_stats[j][1]) + "</td>\n")
                        f.write("  </tr>\n")                   
                    f.write("</table>\n")
                    f.write("<h1></h1>") #Empty header at bottom to ensure bottom of above table is visible

                    f.write('</div>')

                    f.write('<div class="main">\n')

                    asset_name_str = ''
                    if asset_name != '':
                        asset_name_str = ' (' + asset_name + ')'

                    f.write("<h1>" + ET.quantity + " in " + cover_type + muid + asset_name_str + "</h1>\n")

                    for plot in plots:
                        f.write(plot[0])
                        f.write(plot[1].to_html(full_html=False, include_plotlyjs='cdn'))
                    f.write('</div>')
                    f.write('</body>')
                    f.write('</html>')
                f.close()


In [ ]:
output_subfolder

In [ ]:
os.path.basename(output_subfolder)